In [ ]:
reset()

In [ ]:
load('../common_lib.sage')

In [ ]:
## preparser will change / to * backslashOperator()
## if the code is loaded, then it is fine
preparser(False)

In [ ]:
%%writefile ../qbank/CountIntSol.sage

def CountIntSol(param=(5,10), seed=None):
    """
    A quiz generator for counting the number of integer solutions
    
    INPUT:
    
    - ``param`` -- tuple `(num_var, bound)`;
      Consider the equation x1 + ... + xk = w;
      3 <= k <= num_var;
      3 <= w <= bound;
      
    - ``seed`` -- integer;  you may give a seed for  
      the random process if  you like.  Otherwise  
      the default is ``None`` and a random number  
      will be chosen.
      
    OUTPUT:
    
    three strings: QUESTION, SOLUTION, CHECKCODE
    """
    num_var,bound = param
    random.seed(seed)
    
    ans = 0
    while ans <= 10 or ans >= 10000: ### if the ans is too bad
        k = random.randint(3, num_var)
        w = random.randint(3, bound)
        geq = [random.choice([0,1]) for _ in range(k)]
        neq = random.choice(['=', r'\leq'])

        eqn = r"x_1 + \cdots + x_{%s} %s %s"%(k,neq,w)
        conds = ', '.join([r'x_%s\geq %s'%(i+1,geq[i]) for i in range(k)])
        
        up = k if neq == '=' else k+1
        down = w - sum(geq)
        
        ans = binomial(up+down-1, down)
    
    QUESTION = r"""Consider the equation 
\[%(eqn)s\]
under the conditions
\[%(conds)s.\]
Count the number of the integer solutions.

\bigskip
Check code $=$ (sum of all digits of your answer) mod $10$
    """%{'eqn': eqn, 
         'conds': conds, 
        }
    
    y_eqn = r"y_1 + \cdots + y_{%s} = %s"%(up,down)
    trans = ', '.join(['y_{%s} = x_%s %s'%(i+1,i+1,'-1' if geq[i] == 1 else '') 
                       for i in range(k)])
    
    checkcode = sum(int(d) for d in str(ans)) % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""The equivalent equation is 
\[%(y_eqn)s,\]
where 
\[%(trans)s.\]
    
Therefore, the answer is 
\[\binom{%(up)s + %(down)s - 1}{%(down)s} = %(ans)s.\]

\bigskip 
Check code $=$ (sum of all digits of your answer) mod $10$ $= %(checkcode)s$.
    """%({'y_eqn': y_eqn, 
          'trans': trans,
          'up': up,
          'down': down, 
          'ans': ans, 
          'checkcode': checkcode
         })
    
    return QUESTION, SOLUTION, CHECKCODE

In [ ]:
%%writefile ../qbank/Prufer.sage

def Prufer(param=(8,), seed=None):
    """
    A quiz generator for finding the Prufer code of a tree
    
    INPUT:
    
    - ``param`` -- tuple `(tree_order)`; 
      
    - ``seed`` -- integer;  you may give a seed for  
      the random process if  you like.  Otherwise  
      the default is ``None`` and a random number  
      will be chosen.
      
    OUTPUT:
    
    three strings: QUESTION, SOLUTION, CHECKCODE
    """
    tree_order = param[0]
    random.seed(seed)

    g = graphs.RandomTree(tree_order)
    ans = Prufer_code(g)
    code = latex_graph_circular(g)

    QUESTION = r"""Let $T$ be the labeled tree as shown below.

\begin{center}
\begin{tikzpicture}
%(code)s
\end{tikzpicture}
\end{center}

Find the Pr\"ufer code of $T$.

\bigskip
Check code $=$ (sum of all digits of your answer) mod $10$
    """%{'code': code 
        }
    
    compact_ans = ''.join(str(d) for d in ans)
    checkcode = sum(ans) % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""The Pr\"ufer code is $%(compact_ans)s$.

\bigskip 
Check code $=$ (sum of all digits of your answer) mod $10$ $= %(checkcode)s$.
    """%({'compact_ans': compact_ans, 
          'checkcode': checkcode
         })
    
    return QUESTION, SOLUTION, CHECKCODE

In [ ]:
%%writefile ../qbank/RecRel.sage

def RecRel(param=(5,10), seed=None):
    """
    A quiz generator for solving a recurrence relation
    
    INPUT:
    
    - ``param`` -- tuple `(r_bound, c_bound)`;
      the general solution will be 
          c_1 r_1^n  + c_2 r_2^n
      |r_i| <= r_bound
      |c_i| <= c_bound
      
    - ``seed`` -- integer;  you may give a seed for  
      the random process if  you like.  Otherwise  
      the default is ``None`` and a random number  
      will be chosen.
      
    OUTPUT:
    
    three strings: QUESTION, SOLUTION, CHECKCODE
    """
    r_bound, c_bound = param
    random.seed(seed)
    
    c_choices = list(range(-c_bound,c_bound+1))
    c_choices.remove(0)
    c1 = random.choice(c_choices)
    c2 = random.choice(c_choices)

    r_choices = list(range(-r_bound,r_bound+1))
    r_choices.remove(0)
    r_choices.remove(1)
    r1 = random.choice(r_choices)
    r2 = random.choice(r_choices)
    
    g_sol = r"(%s)\cdot(%s)^n + (%s)\cdot(%s)^n"%(c1,r1,c2,r2)
    ak = lambda k: c1 * (r1)**k + c2 * (r2)**k
    a0 = ak(0)
    a1 = ak(1)
    
    p,q = r1+r2, -r1*r2
    rel = r"a_n = (%s) a_{n-1} + (%s) a_{n-2}"%(p,q)
    char_eq = "x^2 = (%s)x + (%s)"%(p,q)

    QUESTION = r"""Solve the recurrence relation
\[%(rel)s \text{ for }n\geq 2,\]
\[a_0 = %(a0)s, a_1 = %(a1)s.\]
Write your solution in the form of 
\[a_n = c_1\cdot r_1^n + c_2\cdot r_2^n\]

\bigskip
Check code $=$ $(c_1+c_2+r_1+r_2)$ mod $10$
    """%{'rel': rel,
         'a0': a0,
         'a1': a1
        }
    
    checkcode = (c1 + c2 + r1 + r2) % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""Solve the characteristic polynomial 
\[%(char_eq)s\]
and get 
\[r_1 = %(r1)s, r_2 = %(r2)s.\]

Then solve the system of linear equations
\[a_0 = c_1 + c_2 = %(a0)s\]
\[a_1 = (%(r1)s) c_1 + (%(r2)s) c_2 = %(a1)s\]
to get 
\[c_1 = %(c1)s, c_2 = %(c2)s.\]

\bigskip 
Check code $=$ $(c_1+c_2+r_1+r_2)$ mod $10$ $= %(checkcode)s$.
    """%({'char_eq': char_eq,
          'r1': r1,
          'r2': r2, 
          'c1': c1, 
          'c2': c2, 
          'a0': a0,
          'a1': a1, 
          'checkcode': checkcode
         })
    
    return QUESTION, SOLUTION, CHECKCODE

In [ ]:
q, s, c = RecRel()

In [ ]:
print(q)
print('---')
print(s)
print('---')
print(c)